In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

def intensity(phi, mask, slm_dim1, slm_dim2, sigma):
    mask2d = mask.reshape(slm_dim1, slm_dim2)
    field = np.exp(1j * 2 * np.pi * phi.reshape(slm_dim1, slm_dim2)) * mask2d
    spec = np.fft.fftshift(np.fft.fft2(field))
    I = np.abs(spec[slm_dim1 // 2, slm_dim2 // 2])**2 / spec.size
    I += sigma * np.random.randn()  # additive Gaussian noise
    return float(I)

def random_mask(slm_dim1, slm_dim2) -> np.ndarray:
    return np.random.choice([0.0, 1.0], size=slm_dim1*slm_dim2).astype(np.float32)

def binary_ga(slm_dim1: int = 16,
              slm_dim2: int = 16,
              pop_size: int = 30,
              generations: int = 200,
              sigma: float = 0.0,
              R0: float = 0.1,
              Rend: float = 0.0025,
              lambda_: float = 50.0,
              phi: np.ndarray | None = None,
              seed: int | None = None):
    
    if seed is not None:
        np.random.seed(seed)

    n_pix = slm_dim1 * slm_dim2
    if phi is None:
        phi = np.random.rand(n_pix).astype(np.float32)

    population = [random_mask(slm_dim1, slm_dim2) for _ in range(pop_size)]
    best_int_history = []
    best_mask = None

    for g in range(1, generations + 1):
        # 1) Evaluate current parents
        intensities = [intensity(phi, m, slm_dim1, slm_dim2, sigma) for m in population]
        order = np.argsort(intensities)[::-1]  # Descending

        # 2) Crossover template
        T_flat = np.zeros(n_pix, dtype=np.float32)
        T_flat[np.random.choice(n_pix, n_pix // 2, replace=False)] = 1.0
        T_breed = T_flat.reshape(slm_dim1, slm_dim2)

        # 3) Generate offspring via crossover
        offspring = []
        for _ in range(pop_size):
            idx_pool = np.random.permutation(pop_size // 2)[:2]
            pa = population[order[idx_pool[0]]].reshape(slm_dim1, slm_dim2)
            ma = population[order[idx_pool[1]]].reshape(slm_dim1, slm_dim2)
            child = pa * T_breed + ma * (1 - T_breed)
            offspring.append(child)

        # 4) Mutate offspring with decaying rate
        mut_rate = (R0 - Rend) * math.exp(-g / lambda_) + Rend
        mut_num = int(round(mut_rate * n_pix))
        mutated = []
        for child in offspring:
            flat = child.flatten()
            flip_idx = np.random.choice(n_pix, mut_num, replace=False)
            flat[flip_idx] = 1.0 - flat[flip_idx]  # Bit‑flip mutation
            mutated.append(flat.reshape(slm_dim1, slm_dim2))

        # 5) Evaluate offspring
        off_int = [intensity(phi, m, slm_dim1, slm_dim2, sigma) for m in mutated]
        off_order = np.argsort(off_int)[::-1]

        # 6) Replacement
        for k in range(pop_size // 2):
            population[order[-(k + 1)]] = mutated[off_order[k]]

        # 7) Logging
        best_parent_int = intensities[order[0]]
        best_off_int = off_int[off_order[0]]
        best_int = max(best_parent_int, best_off_int)

        if best_int == best_parent_int:
            best_mask = population[order[0]]
        else:
            best_mask = mutated[off_order[0]]

        best_int_history.append(best_int)
        print(f"Gen {g:3d}  Best Intensity = {best_int:.4f}")

    return {
        'max_intensity': best_int_history,
        'best_mask': best_mask
    }

phi = np.load("phase_mask/phi_16.npy")
results = binary_ga(slm_dim1=16, slm_dim2=16, pop_size=30, generations=200, sigma=0.01, seed=42, phi=phi)
print(f"Final best intensity: {max(results['max_intensity']):.6f}")

plt.figure(figsize=(16, 16))
plt.plot(results['max_intensity'], label='Best Intensity during Training', color='black')
plt.xlabel('Generation')
plt.ylabel('Intensity')
plt.title('Best Intensity Per Generation')
plt.grid(True)
plt.legend()
plt.savefig('results_ga/binary_ga_training_curve.png')
plt.close()